# MPR121 Data Recording Notebook

Author: Christopher Parker (parkecp@mail.uc.edu)

I've tried to make this as user-friendly as possible, but feel free to reach out with any questions.

### Imports

In [1]:
# Basic libraries
import os
os.environ['BLINKA_MPR121'] = '1'
os.environ['BLINKA_FT232H'] = '1'
import time
import datetime
from collections import deque

# For writing data to file
import h5py

# Libraries for FTDI and I2C
from pyftdi.i2c import I2cController
from pyftdi.usbtools import UsbTools

# Multiprocessing
from concurrent.futures import ThreadPoolExecutor

### Device Configuration and Constant Declarations

In [2]:
devices = UsbTools.find_all([(0x0403, 0x6014)])

In [3]:
i2c_controllers = []
i2c_ports = []
for dev in devices:
    url = f"ftdi://ftdi:232h:{dev[0].address}/1"
    controller = I2cController()
    controller.configure(url)
    i2c_controllers.append(controller)
    port = controller.get_port(0x5A)
    i2c_ports.append(port)

In [7]:
# 0x80 is the Soft Reset Register (0x63 sends reset command)
port.write_to(0x80, b'\x63')
# 0x5E is the configuration register, setting to 8F starts the MPR121
# with the config used in the Adafruit library
port.write_to(0x5E, b'\x8F')

# Now I just need to figure out all of the other configuration options and if
# they ought to be changed

In [6]:
port.write(0x04)
port.read(24)

bytearray(b'\x00\x00\x00\x00\xef\x00\xf3\x00\xef\x00\xf1\x00\xf7\x00\xf6\x00\xfa\x00\xf9\x00\xf8\x00\xf4\x00')

In [53]:
port.write_to(0x5E, b'\x8F')

In [54]:
port.read_from(0x5E, 1)

bytearray(b'\x8f')

In [41]:
port.write(0x5D)
port.read(2)

bytearray(b'\x00\x00')

In [2]:
import adafruit_mpr121
import busio
import board
i2c = busio.I2C(board.SCL, board.SDA)
adafruit_mpr121.MPR121(i2c)

In [ ]:
i2c = busio.I2C(board.SCL, board.SDA, frequency=400000)
# Despite not using this object later, it is necessary to initialize the MPR121
# like this. I'm not sure why, but it just returns 0x00 from all registers
# sometimes, and this seemed to fix it.
mpr121 = adafruit_mpr121.MPR121(i2c)
mpr121_address = 0x5A  # Default I2C address for the MPR121
start_reg = 0x04       # Starting register for filtered (raw) data
raw_buffer = bytearray(24)  # 24 bytes for 12 electrodes (2 bytes each)

# How many sensor samples we want to store before writing
HISTORY_SIZE = 1000


### Widget Setup

#### Recording Function

In [ ]:
def record():
    # Time data (for each channel, because they are updated independently)
    # We use deques here because they are more efficient for appending and popping
    time_data = deque(maxlen=12)
    # Capacitance data (again for each channel)
    cap_data = deque(maxlen=12)
    while not i2c.try_lock():
        pass
    try:
        i2c.writeto_then_readfrom(mpr121_address, bytes([start_reg]), raw_buffer)
    finally:
        i2c.unlock()

    # Process the raw data for each electrode
    for chan in range(12):
        # Combine the two bytes (little-endian) for each electrode
        value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)

        # Save the value to the cap_data list and the current time to the time_data deque
        cap_data.append(value)            
        time_data.append(time.time())
    return time_data, cap_data

## Start Recording Here

In [ ]:
%%time
stop = False # Flag to stop the loop
time_data = deque([deque(maxlen=HISTORY_SIZE) for _ in range(12)])
cap_data = deque([deque(maxlen=HISTORY_SIZE) for _ in range(12)])
# Alternatives for stopping the loop (iterations or time),
# also we are writing to file every 1000 reads
loop_ctr = 0
cutoff_time = time.time() + 60*60*2 # 2 hours
with h5py.File(f"raw_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5", "w") as h5f:
    while not stop:
        start_time = time.time()
        _time_data, _cap_data = record()
        # Append data to the corresponding deques
        for channel_idx, (time_point, cap_point) in enumerate(zip(_time_data, _cap_data)):
            time_data[channel_idx].append(time_point)
            cap_data[channel_idx].append(cap_point)
        loop_time = time.time()
        #print(f"Loop time: {loop_time - start_time}")
        if loop_ctr == 1000:
            print(loop_ctr)
            h5f.create_dataset("time_data", data=time_data, chunks=(12, 1000), maxshape=(12, None))
            h5f.create_dataset("cap_data", data=cap_data, dtype='i2', chunks=(12, 1000), maxshape=(12, None))
        elif loop_ctr != 0 and loop_ctr%1000 == 0:
            tmp_ctr = loop_ctr - 1000
            h5f["time_data"].resize((12, tmp_ctr + HISTORY_SIZE))
            h5f["cap_data"].resize((12, tmp_ctr + HISTORY_SIZE))
            h5f["time_data"][:, tmp_ctr:tmp_ctr + HISTORY_SIZE] = time_data
            h5f["cap_data"][:, tmp_ctr:tmp_ctr + HISTORY_SIZE] = cap_data
            #print(f"Write time: {time.time() - loop_time}")
        # Stop after a set number of reads
        if loop_ctr == 2000:
            stop = True
        if time.time() >= cutoff_time:
            stop = True
        loop_ctr += 1